## Star Trek TNG More Episodes

In [ ]:
import pandas as pd
import re

### Data Is sourced from Github:

#### https://github.com/RTrek/startrekTNGdataset/tree/master/data2

In [ ]:
tng = pd.read_csv("TNG.csv")

In [ ]:
# Modify the dataset for what we will feed into our NLP model
cols = ['who','type', 'text']
tng = tng[cols]
tng.loc[(tng['type'] == 'description') & (tng['who'].isna()), 'who'] = "DESCRIPTION"
tng.drop(['type'], axis=1, inplace=True)

In [ ]:
#Replace all Q characters with 1
tng[tng['who'].str.contains('Q', na=False)] = "Q"

In [ ]:
tng['text'] = tng['who'] + " ::" + tng['text']
tng = tng.rename(columns={'who': 'label'})

tng = tng.dropna(axis=0)

tng['is_valid'] = False

#tng.to_csv('train.csv', index=False)
tng.to_csv('train.csv')

### Read in our Edited Data Frame

In [ ]:
from pathlib import Path
import pandas as pd
import re
import os
from fastai.text import *
import string
import numpy as np

%reload_ext autoreload
%autoreload 2
%matplotlib inline

cwd = os.getcwd()
path = Path(cwd)

### Here is the factory method for implementing our own vocab and tokenizer methods if we choose to do so

In [ ]:
all_letters = list(string.printable + string.whitespace) #Letters need to be in a list not a string

vocab=Vocab.create(all_letters, max_vocab=80000, min_freq=0)
customtokenizer = Tokenizer(pre_rules= [], post_rules=[])

processors = [TokenizeProcessor(),
             NumericalizeProcessor()]

data = (TextList.from_csv(path, 'train.csv', cols=['text'])
                 .split_by_rand_pct(0.01)
                 .label_for_lm()
                 .databunch(bs=192))
data.save('data_block_lm.pkl')

In [ ]:
data_lm = load_data(path, 'data_block_lm.pkl',bs=192)
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)

In [ ]:
data_lm
data_lm.show_batch()

In [ ]:
learn.lr_find()
learn.recorder.plot(skip_end=15)

In [ ]:
learn.fit_one_cycle(20, 1e-2, moms=(0.8,0.7))

In [ ]:
learn.save('fit_head')

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(20, 1e-3, moms=(0.8,0.7))

In [ ]:
learn.save('fine_tuned')
learn.save_encoder('fine_tuned_enc')

In [ ]:
learn.fit_one_cycle(20, 1e-4, moms=(0.8,0.7))

In [ ]:
learn.save('fine_tuned')
learn.save_encoder('fine_tuned_enc')

# We have our trained model Now

In [1]:
from pathlib import Path
import pandas as pd
import re
import os
from fastai.text import *
import string
import numpy as np

%reload_ext autoreload
%autoreload 2
%matplotlib inline

cwd = os.getcwd()
path = Path(cwd)

In [2]:
data_lm = load_data(path, 'data_block_lm.pkl',bs=192)
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.3)
learn.load('fine_tuned')
learn.load_encoder('fine_tuned_enc')

C:\Users\brent\AppData\Roaming\Python\Python37\site-packages\fastai\datasets.py:156: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  with open(fpath, 'r') as yaml_file: return yaml.load(yaml_file)


In [3]:
learn.predict("PICARD", 200, temperature=0.5)

"PICARD : : What is it ? xxbos DESCRIPTION : : Picard , Riker , Worf , Geordi , Troi , Beverly . Troi is sitting on the edge of the bed , Data is at the head of the table , Worf at the other side . The mood is grim . Beverly is standing near the bed , staring at the medical monitor . She is shivering , cold . She looks around the room , feeling a rush of sensation and emotion . xxbos GEORDI : : It 's the best thing that ever happened to you . xxbos PICARD : : What 's the matter ? xxbos DESCRIPTION : : Picard , Riker , Worf , Data , Geordi and Troi are sitting at the table . xxbos DESCRIPTION : : Picard and Riker are in the Command Chair . Worf is seated at his Command Chair . Worf is at Tactical . The N.D 's stand at"

### We Create Some Functions that Will Help us Write 

In [3]:
def GetCharacterLines(character, numberOfLines = 1, temperatureChange = True, temperature = 1.0):
    N_WORDS = 150
    EPSILON = 0.1
    #We predict the next N_WORDS
    #Each iteration increases the temperature of the sentence.  We interpolate between 0 and 1.
    if temperatureChange:
        predictedLines = list(learn.predict(character.upper(), N_WORDS,
                                            temperature=EPSILON + (i/(numberOfLines-1))).split("xxbos")[0] for i in range(numberOfLines))
    else:
        predictedLines = list(learn.predict(character.upper(), N_WORDS,
                                            temperature=temperature).split("xxbos")[0] for i in range(numberOfLines))
    return predictedLines

In [4]:
def RecursiveLine(line = "", numberOfLines = 5, temperature = 1):
    """Accept a line and continuously feeds it into the model to build a block of the script."""
    N_WORDS = 150
    dialog = ""
    for i in range(numberOfLines):
        #Sometimes we don't see the xxbos token and get an exception
        try:
            line = learn.predict(line, N_WORDS, temperature=temperature).split("xxbos")[i] + "xxbos"
        except:
            line = line

        dialog = dialog + line
    
    dialog = dialog.split("xxbos")[:-1]
    return dialog

In [5]:
def ConversationLines(characterList, numberOfLines = 5, temperature = 1):
    """Accept a list of character names.  We reject any sentences that do not begin with these particular characters.
    The same character will not speak 2 times in a row."""
    N_WORDS = 100
    dialog = ""
    line = ""
    lastCharacter = str(None)
    
    characterList = [character.upper() for character in characterList]
    
    for i in range(numberOfLines):
        #Keep creating lines until we have one of the characters
        while True:
            line = ""
            line = learn.predict(line, N_WORDS, temperature=temperature).split("xxbos")[0] + " xxbos "
            #Who is the current speaker for the line?
            currentCharacter = line.split(" ")[1]
            if (any(character in line for character in characterList)) and (currentCharacter != lastCharacter):
                break
    
        dialog = dialog + line
        lastCharacter = line.split(" ")[1]
    
    dialog = dialog.split("xxbos")[:-1]
    dialog = [line.lstrip() for line in dialog]
    return dialog

In [6]:
def ConversationLinesRecursive(characterList, numberOfLines = 5, temperature = 1):
    """Accept a list of character names.  We reject any sentences that do not begin with these particular characters.
    The same character will not speak 2 times in a row.
    Continuously Feeds the past dialog in to generate the text"""
    N_WORDS = 100
    dialog = ""
    lastCharacter = str(None)
    
    #Put the character list into all uppercase
    characterList = [character.upper() for character in characterList]
    
    #We generate the number of lines
    for i in range(numberOfLines):
        
        #Keep creating lines until we have one of the characters
        while True:
            line = learn.predict(dialog, N_WORDS, temperature=temperature).split("xxbos")[-1] + " xxbos "
            #Who is the current speaker for the line?
            currentCharacter = line.split(" ")[1]
            if (any(character in line for character in characterList)) and (currentCharacter != lastCharacter):
                break
    
        dialog = dialog + line
        lastCharacter = line.split(" ")[1]
        
    dialog = dialog.split("xxbos")[:-1]
    dialog = [line.lstrip() for line in dialog]
    return dialog

### Some Examples of Lines we can write automatically

In [23]:
RecursiveLine("Picard")

['Picard : : There are other treasures to be found . ',
 ' DESCRIPTION : : Picard is aghast , as Beverly stands . ',
 " WORF : : Picard must see some depth , even after we accept this new Prime Directive , and do n't know how much time is going to advance . ",
 ' DESCRIPTION : : a beat , then studies her a long beat . ',
 " DESCRIPTION : : This door , off Picard 's shocked reaction ... "]

In [34]:
characters = ["Picard", "Riker", "Counselor"]
ConversationLines(characters, numberOfLines=5, temperature= 0.9)

['RIKER : : An interesting choice .  ',
 "PICARD : : If he looks ... the people in this chamber will have been confined to the brig if you 're to return to your post .  ",
 'RIKER : : Go ahead , Ensign .  ',
 'PICARD : : Darmok .  ',
 'RIKER : : Are you all right ?  ']

In [44]:
characters = ["Picard", "Riker", "Counselor"]
ConversationLinesRecursive(characters, numberOfLines=5, temperature= 0.9)

['PICARD : : Then ',
 "RIKER : : Let 's hook up the ",
 'PICARD : : Begin energy ',
 'RIKER : : Shelby to bridge . We ',
 'PICARD : : Always , you have the knowledge of the future . Your offspring must also be able to repair it . If you wish to happy , i will still be able to guide you into the task which will ']

In [26]:
GetCharacterLines("", numberOfLines=10, temperatureChange=False, temperature=1)

[' DESCRIPTION : : She gives him a appeal and turns away . ',
 ' KEEL : : No word of your orders ... anything unusual . ',
 " DESCRIPTION : : The Cardassians ENTER from Data 's Room and cross close to Beverly . She scans with her tricorder . ",
 " COMPUTER : : The opposing advocate 's job is to find a context in which to explain . ",
 ' DESCRIPTION : : reacts , considering how possible it is , Geordi . ',
 ' PICARD : : Are you saying , i killed manually ... ',
 ' DESCRIPTION : : Taibak hesitates -- he is still monitoring his console . ',
 ' LETEK : : Letek , in the exact same words from Letek , had no choice but to surrender . ',
 ' DESCRIPTION : : Troi smiles and takes a long look ... ',
 ' DATA : : The port engine nacelle has been jolted from the engine . We should not be thrown dead . ']

In [27]:
GetCharacterLines("Picard", 10)

['PICARD : : What is it ? ',
 'PICARD : : What do you mean ? ',
 'PICARD : : What is it ? ',
 "PICARD : : It 's a simple matter , Number One . It 's a very ... Human expression . ",
 "PICARD : : You 're a good officer , Doctor . You can be sure that Lieutenant La Forge is the best officer in the fleet . ",
 'PICARD : : What is the status of the warp engines ? ',
 'PICARD : : i see Spock in his way . ',
 'PICARD : : Captain , you are transmitting nothing . We have succeeded in arming the cell . We have a reason for that -- Commander . ',
 "PICARD 'S COM VOICE : : Captain , this is a Klingon vessel . i 're the first time in ten years for a Klingon to be overly presumptuous . If you may proceed move in your direction , you 've agreed -- ",
 'PICARD : : Captain Picard , any good luck is right . ']

In [28]:
GetCharacterLines("Riker", 10)

["RIKER : : You 're not going to be here . ",
 "RIKER : : We 're going to have to get back to Federation space . ",
 "RIKER : : That 's right . We 're going to have to get to Sickbay . ",
 "RIKER : : We 're going to have to do it all the time . ",
 'RIKER : : Do you have the coordinates to beam down ? ',
 'RIKER : : Not much of a mystery . ',
 'RIKER : : Set course for the Devron System , and engage at warp nine . ',
 "RIKER : : Geordi ... i do n't know what you 're looking for ... ",
 'RIKER : : The two of you had an unusual time . They knew each other for a moment . Wonderful stories ... as well as physical or physical . How did you come on board ? ',
 "RIKER : : That arms like him could fall short of me wide - eyed and i 'd say . "]

In [29]:
GetCharacterLines("Counselor", 10)

["COUNSELOR TROI : : You 're not going to be here for a while . ",
 "COUNSELOR TROI : : Captain , i 'm not sure i can explain it . ",
 "COUNSELOR TROI : : You 're not going to be here , Deanna . ",
 'COUNSELOR TROI : : What is it ? ',
 "COUNSELOR TROI : : You do n't have to talk about it . ",
 'COUNSELOR TROI : : What would you like to hear ? ',
 "COUNSELOR 'S VOICE : : Captain , i have the bridge . Counselor . ",
 "COUNSELOR TROI : : You 're female . ",
 'COUNSELOR TROI COM VOICE : : Troi to Captain Picard . ',
 'COUNSELOR TROI : : After meeting your conscience , in Security ... ']

# Area to Generate Dialog

In [45]:
GetCharacterLines("PICARD ( V.O. ) ", 10)

["PICARD ( V.O. )  : : Captain 's log , supplemental . We have arrived at Starbase Two - twenty - seven . We are in orbit around Beta Agni Two for the Federation and have agreed to contact the Enterprise . ",
 "PICARD ( V.O. )  : : Captain 's log , stardate 45652.1 . The Enterprise has entered an area of space known as the Typhon Expanse . We are the first Starfleet vessel to chart this unexplored region . ",
 "PICARD ( V.O. )  : : Captain 's Log , Stardate 47391.2 . i am returning to the Enterprise in response to a distress call from a Federation science station . It is a distress call from a Federation science station . ",
 "PICARD ( V.O. )  : : Captain 's log , supplemental . We are on a course to Starbase Three- zero - one . We have been on a long , difficult journey to the Neutral Zone . ",
 "PICARD ( V.O. )  : : Captain 's log , Stardate Gemaris Five where , in the meantime , we have been contacted by an alien officer who is en route to the Ramatis Three meeting ... ",
 "PICARD (

In [48]:
GetCharacterLines("DESCRIPTION", 10)

['DESCRIPTION : : The Enterprise is docked at a starbase in a starbase . ',
 'DESCRIPTION : : The Enterprise is struck by the torpedoes from the Klingon ships . ',
 'DESCRIPTION : : The Enterprise is docked at a starbase in a starbase . ',
 'DESCRIPTION : : FADE IN : ',
 'DESCRIPTION : : The voice is quiet . ',
 'DESCRIPTION : : As Riker , Data , Worf , and TWO N.D. SECURITY GUARDS ENTER . Riker ENTERS , looks around the room , sees a small ALIEN TRANSPORTER CREWMEMBER standing near the cell door . He casts a glance over at the Lieutenant . ',
 'DESCRIPTION : : Troi is on her feet , reaching her to come out of the turbolift . ',
 'DESCRIPTION : : They are all looking at each other , looking much more comfortable when their eyes meet . ',
 'DESCRIPTION : : She takes the goblet from the table and sets it on the table ... things boys and drinks are feeling ... something they share in a toast . ',
 'DESCRIPTION : : a beat . ']

In [46]:
RecursiveLine("PICARD ( V.O. ) ",numberOfLines=10)

['PICARD ( V.O. )  : : Our chief medical officer , Lieutenant Commander Riker will attempt to trace the problem , puzzled . ',
 ' q : : q ',
 " PICARD : : i 'd like to work hard enough here to talk . ",
 ' TROI : : No . ',
 ' DESCRIPTION : : The Administrator ignores it troi and confronts Riker . ',
 ' DESCRIPTION : : The motion surrounding proceeded as detailed as we did a moment ago . ',
 ' DESCRIPTION : : But on the way , Geordi steps past Picard and claps him on the shoulder ... ',
 ' DESCRIPTION : : as the ball does not move , the Edolord message is begun again in a description ... ',
 " K'NERA : : i believe it will just be to you and your crew . ",
 " K'NERA : : i believe it will just be to you and your crew . "]

In [8]:
characters = ["RIKER", "DATA", "WORF"]
ConversationLinesRecursive(characters, numberOfLines=20, temperature= 1.0)

['WORF : : Approximately seven ... six years ',
 'DATA : : i am capable of emitting a highly internal reading . However , the secondary systems are very unstable , too ',
 'DESCRIPTION : : Data is seated at Ops . SETAL is in Command , where WORF is functioning at the command stations of his team . Picard and Worf ENTER from transporter room three . Riker enters the turbolift , turns around in his chair , passes a crewperson at his station , sees his ',
 'WORF : : ',
 'DATA : : i am aware of the risks involved . Doctor Marr is very curious about the ',
 'RIKER : : All right they ',
 'TROI RIKER : : What ',
 'DATA : : Thank ',
 'RIKER : ',
 'DATA : : ',
 'WORF : : It is interesting that the Klingons do not understand their studied enemies . ',
 'RIKER : ',
 'DATA : : Your smile fades after ',
 'RIKER : : Mister Worf . Description : : ',
 'DATA : : Geordi , i can promise you a future in human power that would never have been able to sustain your gaze as ',
 'WORF : : Lieutenant ',
 'RIKER

# My Script

PICARD ( V.O. )  : : Captain 's log , supplemental . We have arrived at Starbase Two - twenty - seven . We are in orbit around Beta Agni Two for the Federation and have agreed to contact the Enterprise .

 DESCRIPTION : : As Riker , Data , Worf , and TWO N.D. SECURITY GUARDS ENTER . Riker ENTERS , looks around the room , sees a small ALIEN TRANSPORTER CREWMEMBER standing near the cell door . He casts a glance over at the Lieutenant .
 
 DATA : : i am capable of emitting a highly internal reading . However , the secondary systems are very unstable , too
 
 WORF : : Lieutenant
 
 DATA : : i am aware of the risks involved . Doctor Marr is very curious about the
 
 RIKER : : Captain -- we 're going to need your help .
 
 RIKER : : That 's right . The Romulans have crossed into the Neutral Zone . You 've taken a weapon from Khitomer . They 've told me you 've failed in their mission .
 
 PICARD : : It 's not why you came here . When i was a boy
 
 RIKER : 
 
 PICARD : : Geordi ... we 're in the midst of a Red Alert . You have to leave now . If you
 
 RIKER : : They 're waiting for us . You
 
 PICARD : : The Federation 's finest specialist in communication with Alpha state , and who has unique skills ... and eight hours of stellar physics . We will be able to determine that this Beta Stromgren system will be following this action with
 
 PICARD : : Mister Worf
 
 RIKER : : But
 
 PICARD : : Worf , what
 
 RIKER : : i 'm not sure . i think there 's a lot of interference from the Romulans . We 've tried to shut it down -- but we 're not going
 
 PICARD : : Mister Worf ... do you have any idea what the Duras

 RIKER : : No . You were Starfleet 's finest specialist in communication with Cardassian

